## premier test avec pytorch et premier hook

In [1]:
import requests
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import torch.nn.functional as F
import torch

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "gpt2-large"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token =  tokenizer.eos_token
#tokenizer.padding_side = "left"

In [4]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3840, nx=1280)
          (c_proj): Conv1D(nf=1280, nx=1280)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=5120, nx=1280)
          (c_proj): Conv1D(nf=1280, nx=5120)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)

In [4]:
def output_hook(module, input, output):
    print(f'{module} : output')
    print(output.shape)
    print(output)
def input_hook(module, input, output):
    print(f'{module} : input')
    print(input)

def naive_noise_hook(module,input,output): #a rajouter : le fait de selectionner uniquement les bon token, et la bonne variance pour le bruit
    noise = torch.randn_like(output)#*sqrt(3*variance)
    return output+noise

In [43]:
# hook pour observer ce qu'il se passe dedans
hook1 = model.transformer.wte.register_forward_hook(input_hook)
hook2 = model.transformer.wte.register_forward_hook(output_hook)
hook3 = model.transformer.wpe.register_forward_hook(input_hook)
hook4 = model.transformer.wpe.register_forward_hook(output_hook)
hook5 = model.transformer.drop.register_forward_hook(input_hook)
hook6 = model.transformer.drop.register_forward_hook(output_hook)


In [8]:
#hook pour ajouter du bruit
noise_hook1 = model.transformer.drop.register_forward_hook(naive_noise_hook)

In [ ]:
# deuxième run avec bruit
input= tokenizer("Audible.com is owned by", return_tensors="pt")
with torch.no_grad():
    outputs = model(**input, labels = input.input_ids, output_hidden_states = True, output_attentions =True)
probs = F.softmax(outputs.logits[0, -1, :], dim=-1)
top_probs, top_indices = torch.topk(probs, 4)

# les probas des mots
top_words = [tokenizer.decode([idx]) for idx in top_indices]
for word, prob in zip(top_words, top_probs):
    print(f"{word}: {prob.item():.4f}")

In [10]:
hook1.remove()
hook2.remove()
hook3.remove()
hook4.remove()
hook5.remove()
hook6.remove()
noise_hook1.remove()

## dit si un token se réfère au sujet

In [3]:
url = 'https://rome.baulab.info/data/dsets/known_1000.json'
response = requests.get(url) 
data = response.json()

In [4]:
prompts = [dict['prompt'] for dict in data]
subjects = [dict['subject'] for dict in data]
input= tokenizer(prompts, return_tensors="pt", padding= True, return_offsets_mapping= True)

In [5]:
mask = []
for j, prompt in enumerate(prompts):
    map = torch.zeros_like(input.input_ids[j], dtype=torch.int)
    for i,t in enumerate(input.offset_mapping[j]):
        
        if (prompts[j].find(subjects[j])-1<=t[0]) and (t[1]<=prompts[j].find(subjects[j])+len(subjects[j])):
            map[i] = 1
    mask.append(map)
masks_tensor = torch.stack(mask)
masks_tensor = torch.logical_and(masks_tensor, input.attention_mask).int()
masks_tensor

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]], dtype=torch.int32)

pour le prompt i, masks-tensor[i] donne un mask qui dit si oui ou non les tokens se réfèrent au sujet (1 si c'est le cas, 0 sinon)

## rajoute le bruit sur les bons tokens

In [6]:
def noise_hook(module,input,output):
    noise = torch.randn_like(output)*5
    noisy_output = output + noise * masks_tensor.unsqueeze(-1).float()
    print(output.shape, masks_tensor.shape)

    return noisy_output

In [7]:
# fonction qui sert à obtenir le logits du dernier non-padding token
def last_non_padding_token_logits(logits, attention_mask):
    # For each input, find the last non-padding token
    last_non_padding_logits = []
    
    for i in range(logits.size(0)):  # Loop over each prompt in the batch
        # Find the last non-padding token position
        non_padding_positions = (attention_mask[i] == 1).nonzero(as_tuple=True)[0]
        last_non_padding_token_index = non_padding_positions[-1]
        
        # Get the logits of the last non-padding token
        last_non_padding_logits.append(logits[i, last_non_padding_token_index])
    last_non_padding_logits = torch.stack(last_non_padding_logits)
    return last_non_padding_logits

sans le bruit

In [7]:
#sans le noise
input= tokenizer(prompts, return_tensors="pt", padding= True)
with torch.no_grad():
    outputs = model(**input, output_hidden_states = True, output_attentions =True)



`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [8]:
logits_utile = last_non_padding_token_logits(outputs.logits,input.attention_mask)

#pour le prompt ...
probs = F.softmax(logits_utile, dim=-1)
top_probs, top_indices = torch.topk(probs[0], 5)

# les probas des mots
top_words = [tokenizer.decode([idx]) for idx in top_indices]
for word, prob in zip(top_words, top_probs):
    print(f"{word}: {prob.item():.4f}")

 Antarctica: 0.0979
 G: 0.0524
 North: 0.0484
 South: 0.0466
 Africa: 0.0264


avec le bruit cette fois : 

In [8]:
masked_noise_hook = model.transformer.drop.register_forward_hook(noise_hook)

input= tokenizer(prompts, return_tensors="pt", padding= True)
with torch.no_grad():
    outputs = model(**input, labels = input.input_ids, output_hidden_states = True, output_attentions =True)



torch.Size([1209, 27, 1280]) torch.Size([1209, 27])


`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [9]:
logits_utile = last_non_padding_token_logits(outputs.logits,input.attention_mask)

#Le mot prédit pour le 1er prompt avec le noise
probs = F.softmax(logits_utile, dim=-1)
top_probs, top_indices = torch.topk(probs[0], 4)

# les probas des mots
top_words = [tokenizer.decode([idx]) for idx in top_indices]
for word, prob in zip(top_words, top_probs):
    print(f"{word}: {prob.item():.4f}")

 the: 0.4232
 a: 0.0520
 of: 0.0176
 its: 0.0138
